In [1]:
import cv2
import os
import shutil
import numpy as np
import zipfile
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from PIL import Image, ImageFilter, ImageEnhance
from keras.optimizers import Adam
from keras.regularizers import l2

In [3]:
# Unzipping the image dataset
zip_ref = zipfile.ZipFile('original.zip', 'r')
zip_ref.extractall('dataset')  # Extracting to the 'dataset' folder
zip_ref.close()

In [4]:
# blurred
datagen = ImageDataGenerator()


blurred_dir = 'dataset/blurred'
if not os.path.exists(blurred_dir):
    os.makedirs(blurred_dir)


original_dir = 'dataset/original'
total_images = 20


for i in range(1, total_images + 1):
    img_path = f'{original_dir}/{i:02d}.jpg'
    img = Image.open(img_path)

    if img.mode != 'RGB':
        img = img.convert('RGB')

    for j in range(10):
        blurred_img = img.filter(ImageFilter.GaussianBlur(radius=5))
        blurred_img.save(f'{blurred_dir}/blurred_{i:02d}_{j}.jpg')

In [5]:
# Define data paths
original_dir = 'dataset/original'
blurred_dir = 'dataset/blurred'
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'

# Create directories for training and validation
for category in ['original', 'blurred']:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, category), exist_ok=True)

# Function to split data and move files
def split_data(source_folder, train_folder, validation_folder, validation_split=0.2):
    files = os.listdir(source_folder)
    train_files, validation_files = train_test_split(files, test_size=validation_split)

    # Copy files to training directory
    for file in train_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(train_folder, file))

    # Copy files to validation directory
    for file in validation_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(validation_folder, file))

# Apply function to original and distorted datasets
split_data(original_dir, os.path.join(train_dir, 'original'), os.path.join(validation_dir, 'original'))
split_data(blurred_dir, os.path.join(train_dir, 'blurred'), os.path.join(validation_dir, 'blurred'))

In [6]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    'dataset/validation',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

Found 176 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


In [12]:
import shutil

def remove_unwanted_dirs(directory):
    checkpoint_dir = os.path.join(directory, '.ipynb_checkpoints')
    if os.path.exists(checkpoint_dir):
        shutil.rmtree(checkpoint_dir)
        print(f"Removed {checkpoint_dir}")

remove_unwanted_dirs(train_dir)
remove_unwanted_dirs(validation_dir)

print("Updated training directories:", os.listdir(train_dir))
print("Updated validation directories:", os.listdir(validation_dir))

Updated training directories: ['original', 'blurred']
Updated validation directories: ['original', 'blurred']


In [13]:
dataset_dir = 'dataset'
print("Directories in dataset:", os.listdir(dataset_dir))

for category in os.listdir(dataset_dir):
    category_path = os.path.join(dataset_dir, category)
    if os.path.isdir(category_path):
        print(f"Contents of {category}:", os.listdir(category_path))

Directories in dataset: ['validation', 'train', 'blurred']
Contents of validation: ['original', 'blurred']
Contents of train: ['original', 'blurred']
Contents of blurred: ['blurred_04_8.jpg', 'blurred_11_4.jpg', 'blurred_09_5.jpg', 'blurred_15_0.jpg', 'blurred_01_0.jpg', 'blurred_18_5.jpg', 'blurred_02_6.jpg', 'blurred_09_8.jpg', 'blurred_19_1.jpg', 'blurred_11_9.jpg', 'blurred_04_4.jpg', 'blurred_20_0.jpg', 'blurred_14_8.jpg', 'blurred_01_2.jpg', 'blurred_18_9.jpg', 'blurred_06_0.jpg', 'blurred_13_5.jpg', 'blurred_09_1.jpg', 'blurred_06_8.jpg', 'blurred_06_1.jpg', 'blurred_05_0.jpg', 'blurred_17_4.jpg', 'blurred_19_2.jpg', 'blurred_14_6.jpg', 'blurred_15_6.jpg', 'blurred_02_3.jpg', 'blurred_10_4.jpg', 'blurred_18_8.jpg', 'blurred_20_2.jpg', 'blurred_16_6.jpg', 'blurred_12_7.jpg', 'blurred_19_7.jpg', 'blurred_20_1.jpg', 'blurred_13_7.jpg', 'blurred_01_1.jpg', 'blurred_04_0.jpg', 'blurred_17_0.jpg', 'blurred_08_7.jpg', 'blurred_11_7.jpg', 'blurred_12_3.jpg', 'blurred_09_7.jpg', 'blurred

In [14]:
import shutil
import os

directories_to_remove = ['original', 'blurred']
base_dir = 'dataset'

for dir_name in directories_to_remove:
    dir_path = os.path.join(base_dir, dir_name)
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print(f"Removed directory: {dir_path}")
    else:
        print(f"Directory does not exist, no need to remove: {dir_path}")

print("Updated contents of the dataset directory:", os.listdir(base_dir))

Directory does not exist, no need to remove: dataset/original
Removed directory: dataset/blurred
Updated contents of the dataset directory: ['validation', 'train']


In [15]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)  # L2
x = Dropout(0.5)(x)  # Dropout
predictions = Dense(1, activation='sigmoid')(x)

optimizer = tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.001)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Prepare training data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2  # Use 20% of the data as validation set
)

train_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='validation'
)

# Use early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

Found 177 images belonging to 2 classes.
Found 43 images belonging to 2 classes.


In [16]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=15,
    epochs=10,
    callbacks=[early_stopping],
    validation_data=validation_generator,
    validation_steps=5
)

# Evaluate the model
results = model.evaluate(validation_generator)
print("Test loss, Test accuracy:", results)

Epoch 1/10
15/15 [==============================] - 67s 4s/step - loss: 5.9644 - accuracy: 0.7639 - val_loss: 5.8242 - val_accuracy: 0.8800
Epoch 2/10
15/15 [==============================] - 65s 4s/step - loss: 9.9256 - accuracy: 0.7200 - val_loss: 4.9605 - val_accuracy: 0.7600
Epoch 3/10
15/15 [==============================] - 65s 4s/step - loss: 6.0917 - accuracy: 0.7200 - val_loss: 6.1995 - val_accuracy: 0.6800
Epoch 4/10
15/15 [==============================] - 64s 4s/step - loss: 8.1063 - accuracy: 0.6400 - val_loss: 12.0477 - val_accuracy: 0.6400
Epoch 5/10
15/15 [==============================] - 66s 5s/step - loss: 8.9064 - accuracy: 0.6267 - val_loss: 11.6108 - val_accuracy: 0.6800
Epoch 6/10
15/15 [==============================] - 64s 4s/step - loss: 6.8776 - accuracy: 0.7500 - val_loss: 4.6000 - val_accuracy: 0.8000
Epoch 7/10
15/15 [==============================] - 64s 4s/step - loss: 5.5423 - accuracy: 0.7500 - val_loss: 8.0585 - val_accuracy: 0.7200
Epoch 8/10
15/15 [